In [30]:
import pandas as pd
import requests as rq
import time
import os

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

In [31]:
#에브리타임 자동 로그인
user_id = 'baejh0301'
user_pw = 'vkfks77skql97!'

options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
driver.maximize_window()

URL = 'https://everytime.kr'

driver.get(URL)
time.sleep(1)

login_btn = driver.find_element_by_xpath('/html/body/aside/div[1]/a[2]')
login_btn.click()
time.sleep(1)

input_id = driver.find_element_by_xpath('//*[@id="container"]/form/p[1]/input').send_keys(user_id)
input_pw = driver.find_element_by_xpath('//*[@id="container"]/form/p[2]/input').send_keys(user_pw)

time.sleep(1)

enter_btn = driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input')
enter_btn.click()

In [32]:
lec_URL = "https://everytime.kr/lecture"
driver.get(lec_URL)

body = driver.find_element_by_css_selector('body')
for i in range(10):
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.25)

In [33]:
board_html = driver.page_source
board_bs = bs(board_html, 'html.parser')

review_list = list()
star_list = list()
pos_mid_neg_list = list()

review = board_bs.find_all("p", {"class":"text"})
for rev in review:
    review_list.append(rev.text)
    
star = board_bs.find_all("span", {"class":"star"})
for s in star:
    rating = str(s)[50:52]
    if   rating == "10":
        star_list.append(5)
        pos_mid_neg_list.append("pos")
    elif rating == "80":
        star_list.append(4)
        pos_mid_neg_list.append("pos")
    elif rating == "60":
        star_list.append(3)
        pos_mid_neg_list.append("mid")
    elif rating == "40":
        star_list.append(2)
        pos_mid_neg_list.append("neg")
    elif rating == "20":
        star_list.append(1)
        pos_mid_neg_list.append("neg")        

In [34]:
reviews = pd.DataFrame({
    'review': review_list,
    'star': star_list,
    'pos_neg': pos_mid_neg_list})

In [35]:
reviews.head()

,review,star,pos_neg
0,제일 좋았던 수업.. 시험공부는 강의 교안 보면서 개념 정리했더니 다 아는 문제 나...,5,pos
1,온오프 선택이라서 원하는대로 들을 수 있습니다시험은 별로 어렵지는 않고 수업 열심히...,5,pos
2,소설가 구보씨가 환생한줄 알았어요. 의식의 흐름 기법 장인이십니다. 뭘 배운다는 느...,2,neg
3,그냥저냥 무난하게 들을 만한 고전명저북클럽 강의책 안 좋아하면 힘들듯,3,mid
4,대학원 수업이라 저기의 졸업존문을 주체 맞게 논문을 쓰고 발표하면 됩니다.,5,pos


In [28]:
if not os.path.exists('lec_review.csv'):
    reviews.to_csv('lec_review.csv', index=False, mode='w', encoding='utf-8-sig')
else:
    reviews.to_csv('lec_review.csv', index=False, mode='a', encoding='utf-8-sig', header=False)